# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

# Import Libraries

In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 50 #experiment with the batch size

# Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
img_idx = range(0,30,1) #create a list of image numbers you want to use for a particular video
x=len(img_idx)
y=128
z=128

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    image=resize(image,(y,z))
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255. #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255. #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255. #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        batch_data = np.zeros((len(folder_list)%batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((len(folder_list)%batch_size,5)) # batch_labels is the one hot representation of the output    
        for folder in range(len(folder_list)%batch_size):
          imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
          for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
              image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
              
              #crop the images and resize them. Note that the images are of 2 different shape 
              #and the conv3D will throw error if the inputs in a batch have different shapes
              
              image=resize(image,(y,z))
              batch_data[folder,idx,:,:,0] = image[:,:,0]/255. #normalise and feed in the image
              batch_data[folder,idx,:,:,1] = image[:,:,1]/255. #normalise and feed in the image
              batch_data[folder,idx,:,:,2] = image[:,:,2]/255. #normalise and feed in the image
                    
              
          batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
        yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 25 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 25


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [6]:
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout,GlobalAveragePooling2D
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from tensorflow.keras.applications import InceptionV3

#write your model here

base_model = InceptionV3(weights='imagenet',input_shape=(y,z,3),include_top=False)
base_model.trainable = False

model = Sequential()
model.add(TimeDistributed(base_model,input_shape=(len(img_idx),y,z,3)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(GlobalAveragePooling2D()))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Dropout(0.5)))
model.add(LSTM(128,dropout=0.5))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

2022-07-08 17:59:25.833241: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-07-08 17:59:25.833310: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14803 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:1d:00.0, compute capability: 7.5


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [7]:
optimiser = tf.keras.optimizers.Adam(lr=0.01) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 30, 2, 2, 2048)   21802784  
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 30, 2, 2, 2048)   8192      
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 30, 2, 2, 2048)   0         
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 30, 2048)         0         
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 30, 2048)         8

/usr/local/lib/python3.8/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [8]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [9]:
!rm -rf model

In [10]:
# model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
model_name = 'model/'

if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:02d}-{categorical_accuracy:.2f}-{val_categorical_accuracy:.2f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')

LR = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=2, min_lr=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [12]:
import warnings
warnings.filterwarnings('ignore')

history=model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 50
Epoch 1/25


2022-07-08 17:59:46.855174: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


14/14 [==============================] - ETA: 0s - loss: 1.8488 - categorical_accuracy: 0.3575Source path =  /home/datasets/Project_data/val ; batch size = 50

Epoch 00001: val_categorical_accuracy improved from -inf to 0.41000, saving model to model/model-01-0.36-0.41.h5
14/14 [==============================] - 161s 11s/step - loss: 1.8488 - categorical_accuracy: 0.3575 - val_loss: 1.3886 - val_categorical_accuracy: 0.4100 - lr: 0.0100
Epoch 2/25
14/14 [==============================] - ETA: 0s - loss: 1.2378 - categorical_accuracy: 0.5294 
Epoch 00002: val_categorical_accuracy did not improve from 0.41000
14/14 [==============================] - 149s 11s/step - loss: 1.2378 - categorical_accuracy: 0.5294 - val_loss: 1.3362 - val_categorical_accuracy: 0.4000 - lr: 0.0100
Epoch 3/25
14/14 [==============================] - ETA: 0s - loss: 1.1698 - categorical_accuracy: 0.5551 
Epoch 00003: val_categorical_accuracy improved from 0.41000 to 0.43000, saving model to model/model-03-0.56-0.

In [13]:
print(history.history['val_categorical_accuracy'])

[0.4099999964237213, 0.4000000059604645, 0.4300000071525574, 0.5699999928474426, 0.550000011920929, 0.6299999952316284, 0.6200000047683716, 0.6200000047683716, 0.6200000047683716, 0.6399999856948853, 0.6499999761581421, 0.6600000262260437, 0.6299999952316284, 0.6200000047683716, 0.6399999856948853, 0.6299999952316284, 0.6299999952316284, 0.6399999856948853, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6600000262260437, 0.6600000262260437, 0.6700000166893005]


In [ ]:
import warnings
warnings.filterwarnings('ignore')

history2=model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=50, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=26)

Epoch 27/50
14/14 [==============================] - ETA: 0s - loss: 0.5470 - categorical_accuracy: 0.7979
Epoch 00027: val_categorical_accuracy improved from 0.67000 to 0.68000, saving model to model/model-27-0.80-0.68.h5
14/14 [==============================] - 147s 11s/step - loss: 0.5470 - categorical_accuracy: 0.7979 - val_loss: 0.9541 - val_categorical_accuracy: 0.6800 - lr: 1.0000e-04
Epoch 28/50
14/14 [==============================] - ETA: 0s - loss: 0.4921 - categorical_accuracy: 0.8190
Epoch 00028: val_categorical_accuracy did not improve from 0.68000
14/14 [==============================] - 149s 11s/step - loss: 0.4921 - categorical_accuracy: 0.8190 - val_loss: 0.9553 - val_categorical_accuracy: 0.6800 - lr: 1.0000e-04
Epoch 29/50
 9/14 [==================>...........] - ETA: 51s - loss: 0.5452 - categorical_accuracy: 0.7956 